In [1]:
import sys
sys.path.append('d://the-recommender/DNN_for_YouTube_Recommendations')

from sample import ml1m_sample_tfrecord
from core.trainner.trainner_cpu import TrainnerCPU
import tensorflow as tf  
from  infer import Inference
from datasets.ml1m_dataset import Ml1mDataset 



In [2]:
##初始化参数。在main.py中使用argparse包接收解析参数，此处为了方便在jupyter中演示，将参数放在字典中。
parsed_args = {}
#解析dataset参数
parsed_args["data_path"] =  'd://the-recommender/DNN_for_YouTube_Recommendations/datasets/ml-1m/'
parsed_args["users_file"] =  'users.dat'
parsed_args["ratings_file"] =  'ratings.dat' 
parsed_args["movies_file"] =  'movies.dat'
parsed_args["sep"] =  '::'
parsed_args["header"] =  None

#解析样本参数
parsed_args["sample_output_file"] =  'd://the-recommender/DNN_for_YouTube_Recommendations/sample/__TFRecord/train.tfrecord' 

#解析模型保存参数
parsed_args["export_path"] =  'd://the-recommender/DNN_for_YouTube_Recommendations/SAVEDMODELS/savedmodel/default_signature_with_multiple_input_output'
parsed_args["export_path_multi"] = 'd://the-recommender/DNN_for_YouTube_Recommendations/SAVEDMODELS/savedmodel/multi_signature_with_multiple_input_output'

#解析特征hash参数
parsed_args["userid_hash_nums"] = 10000
parsed_args["itemid_hash_nums"] = 10000
parsed_args["zipid_hash_nums"] = 10000
parsed_args["gender_hash_nums"] = 10
parsed_args["age_hash_nums"] = 100
parsed_args["occupation_hash_nums"] = 100
parsed_args["seqlen_hash_nums"] = 100
parsed_args["hash_salt"] = 1024
parsed_args["vocab_size"] = 1000
parsed_args["seq_len"] = 5

#解析模型网络结构参数
parsed_args["userid_input_dim"] = 1
parsed_args["movieid_input_dim"] = 1
parsed_args["rated_movies_lastN_input_dim"] = 5
parsed_args["genres_input_dim"] = 6
parsed_args["zip_id_input_dim"] = 1
parsed_args["num_ratings"] = 6 
parsed_args["dense_size"] = 256    
parsed_args["embedding_out_dim"] = 64

#解析模型训练参数
parsed_args["batch_size"] = 64
parsed_args["epochs"] = 20 
parsed_args["validation_split"] = 0.2       
parsed_args["score_loss_weight"] = 1
parsed_args["rating_loss_weight"] = 1 
parsed_args["learning_rate"] = 1e-3

In [3]:
#加载数据集
data_loader = Ml1mDataset()
data_X, data_y = data_loader.load_ml1m(parsed_args)

In [4]:
#构造样本

#初始化
trainner = TrainnerCPU()
sample_num = 20000

#从原始数据集构造样本,并转换为tfrecord格式
train_dataset = ml1m_sample_tfrecord.Ml1mTFRecordSample().tfrecord_sample(data_X[-sample_num:], data_y[-sample_num:], True, parsed_args) 

##直接从tfrecord文件读取样本
#train_dataset = ml1m_sample_tfrecord.Ml1mTFRecordSample().tfrecord_sample(parsed_args) 

count = train_dataset.take(-1).reduce(tf.constant(0), lambda x, _: x + 1).numpy()  
count


8256

In [5]:
## 进行特征处理, 构造输入tensor.(只是用于演示，在trainner_cpu.py内部已经调用了该方法)
input_tensor_dict = trainner.build_input_tensor(train_dataset,parsed_args)
input_tensor_dict

{'onehot_rating_label_tensor': <tf.Tensor: shape=(8256, 6), dtype=float32, numpy=
 array([[0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        ...,
        [0., 1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.]], dtype=float32)>,
 'rating_score_tensor': <tf.Tensor: shape=(8256,), dtype=float32, numpy=array([4., 5., 5., ..., 4., 4., 3.], dtype=float32)>,
 'hashed_userid_tensor': <tf.Tensor: shape=(8256,), dtype=float32, numpy=array([3995., 3995., 3995., ..., 6493., 6493., 6493.], dtype=float32)>,
 'hashed_itemid_tensor': <tf.Tensor: shape=(8256,), dtype=float32, numpy=array([3323., 4507., 7644., ..., 2781., 5212., 1980.], dtype=float32)>,
 'hashed_zip_tensor': <tf.Tensor: shape=(8256,), dtype=float32, numpy=array([2035., 2035., 2035., ..., 3687., 3687., 3687.], dtype=float32)>,
 'tags_tensor': <tf.Tensor: shape=(8256, 310), dtype=float32, numpy=
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 

In [8]:
##获取推理器
infer = Inference()
signatures = infer.get_infer(parsed_args["export_path_multi"])  #预测结果推理器和特征推理器
signatures

_SignatureMap({'serving_default': <ConcreteFunction (*, genres: TensorSpec(shape=(None, 6), dtype=tf.float32, name='genres'), movie_id: TensorSpec(shape=(None, 1), dtype=tf.float32, name='movie_id'), rated_movies_lastN: TensorSpec(shape=(None, 5), dtype=tf.float32, name='rated_movies_lastn'), tags: TensorSpec(shape=(None, 310), dtype=tf.float32, name='tags'), user_id: TensorSpec(shape=(None, 1), dtype=tf.float32, name='user_id'), zip_id: TensorSpec(shape=(None, 1), dtype=tf.float32, name='zip_id')) -> Dict[['output_1', TensorSpec(shape=(None, 6), dtype=tf.float32, name='output_1')], ['output_0', TensorSpec(shape=(None, 1), dtype=tf.float32, name='output_0')]] at 0x24FC1EDBD50>, 'predict': <ConcreteFunction (*, genres: TensorSpec(shape=(None, 6), dtype=tf.float32, name='genres'), movie_id: TensorSpec(shape=(None, 1), dtype=tf.float32, name='movie_id'), rated_movies_lastN: TensorSpec(shape=(None, 5), dtype=tf.float32, name='rated_movies_lastn'), tags: TensorSpec(shape=(None, 310), dtype=

In [9]:
serve_signature_infer = signatures["predict"]
extract_features_signature_infer = signatures["extract_features"]

In [10]:
#预测结果推理器（分类和回归）
serve_signature_infer

<ConcreteFunction (*, genres: TensorSpec(shape=(None, 6), dtype=tf.float32, name='genres'), movie_id: TensorSpec(shape=(None, 1), dtype=tf.float32, name='movie_id'), rated_movies_lastN: TensorSpec(shape=(None, 5), dtype=tf.float32, name='rated_movies_lastn'), tags: TensorSpec(shape=(None, 310), dtype=tf.float32, name='tags'), user_id: TensorSpec(shape=(None, 1), dtype=tf.float32, name='user_id'), zip_id: TensorSpec(shape=(None, 1), dtype=tf.float32, name='zip_id')) -> Dict[['output_1', TensorSpec(shape=(None, 6), dtype=tf.float32, name='output_1')], ['output_0', TensorSpec(shape=(None, 1), dtype=tf.float32, name='output_0')]] at 0x24FC1EC3150>

In [11]:
#特征推理器
extract_features_signature_infer

<ConcreteFunction (*, genres: TensorSpec(shape=(None, 6), dtype=tf.float32, name='genres'), movie_id: TensorSpec(shape=(None, 1), dtype=tf.float32, name='movie_id'), rated_movies_lastN: TensorSpec(shape=(None, 5), dtype=tf.float32, name='rated_movies_lastn'), tags: TensorSpec(shape=(None, 310), dtype=tf.float32, name='tags'), user_id: TensorSpec(shape=(None, 1), dtype=tf.float32, name='user_id'), zip_id: TensorSpec(shape=(None, 1), dtype=tf.float32, name='zip_id')) -> Dict[['output_0', TensorSpec(shape=(None, 64), dtype=tf.float32, name='output_0')]] at 0x24FC1EA1D90>

In [12]:
## 评分结果推理 - 包括分类和回归
predict_output = serve_signature_infer(user_id=input_tensor_dict["hashed_userid_tensor"], 
                                 movie_id=input_tensor_dict["hashed_itemid_tensor"],
                                 rated_movies_lastN=input_tensor_dict["paded_itemid_lastN_tensor"],
                                 genres=input_tensor_dict["paded_genres_tensor"],
                                 zip_id=input_tensor_dict["hashed_zip_tensor"],
                                 tags=input_tensor_dict["tags_tensor"])  
predict_output

{'output_1': <tf.Tensor: shape=(8256, 6), dtype=float32, numpy=
 array([[-10.418308  ,  -0.5688839 ,  -1.6238482 ,  -1.2239031 ,
          -2.5874405 ,  -2.5621088 ],
        [-13.03817   ,  -1.2424299 ,  -0.68186384,  -0.37645024,
          -1.5695287 ,  -3.783773  ],
        [-17.22326   ,  -1.6669496 ,  -2.308467  ,  -2.0910754 ,
          -3.9598432 ,  -3.3983982 ],
        ...,
        [-11.637632  ,  -1.1239374 ,  -0.09272848,  -3.4240985 ,
          -0.75499153,  -1.7070343 ],
        [-11.140631  ,  -0.36762413,  -0.79162693,  -2.0439606 ,
          -2.225284  ,  -3.7421157 ],
        [-13.236862  ,  -1.1861843 ,  -0.99328804,  -1.6223104 ,
          -3.1108303 ,  -5.5107045 ]], dtype=float32)>,
 'output_0': <tf.Tensor: shape=(8256, 1), dtype=float32, numpy=
 array([[4.031341 ],
        [3.5402346],
        [4.4145136],
        ...,
        [2.8114688],
        [3.7264738],
        [3.7082283]], dtype=float32)>}

In [13]:

#取rating分类预测结果
lable_predictions = tf.argmax(predict_output["output_1"], axis=1)  
lable_predictions

<tf.Tensor: shape=(8256,), dtype=int64, numpy=array([1, 3, 1, ..., 2, 1, 2], dtype=int64)>

In [14]:
print(lable_predictions.numpy().max())
print(lable_predictions.numpy().min())
print(lable_predictions.numpy().mean())

5
1
2.0081153100775193


In [15]:
#取rating回归预测结果
score_predictions = predict_output["output_0"] 
score_predictions

<tf.Tensor: shape=(8256, 1), dtype=float32, numpy=
array([[4.031341 ],
       [3.5402346],
       [4.4145136],
       ...,
       [2.8114688],
       [3.7264738],
       [3.7082283]], dtype=float32)>

In [16]:

## 取最后一层特征结果推理 
feature_output = extract_features_signature_infer(user_id=input_tensor_dict["hashed_userid_tensor"], 
                                 movie_id=input_tensor_dict["hashed_itemid_tensor"],
                                 rated_movies_lastN=input_tensor_dict["paded_itemid_lastN_tensor"],
                                 genres=input_tensor_dict["paded_genres_tensor"],
                                 zip_id=input_tensor_dict["hashed_zip_tensor"],
                                 tags=input_tensor_dict["tags_tensor"])  
feature_output["output_0"]

<tf.Tensor: shape=(8256, 64), dtype=float32, numpy=
array([[0.        , 0.        , 0.18533131, ..., 0.63260573, 0.        ,
        0.53615534],
       [0.17976993, 0.        , 0.41525725, ..., 0.71256757, 0.        ,
        0.27743727],
       [0.        , 0.        , 0.77123106, ..., 0.64668447, 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.28179365, ..., 0.24774575, 0.02933156,
        0.        ],
       [0.2591001 , 0.        , 0.41225904, ..., 1.1945828 , 0.        ,
        0.        ],
       [0.05773006, 0.        , 0.87651336, ..., 1.3249736 , 0.        ,
        0.        ]], dtype=float32)>